# Vgg16 구조를 가진 DFL-CNN 학습

### 모듈 import

In [1]:
import os, sys, cv2, glob, wandb
import dfl_model.dfl_cnn as dfl_model
import numpy as np
import torchvision, torch
import torch.optim as optim
import torchmetrics as metrics
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

### torch 버전 확인하기

In [3]:
print(f'torch:', torch.__version__)
print(f'torchvision:', torchvision.__version__)

torch: 1.13.1
torchvision: 0.14.1


### SEED 고정

In [4]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-2
batch_size = 16
num_epoch = 100

### 데이터셋 불러오기

In [5]:
# train 데이터셋 불러오기

train_path = '/aiffel/aiffel/final_project/dataset/fine_grained_data_tree/train'

# ImageFolder를 이용해 폴더명으로 class 지정
train_dataset = ImageFolder(root=train_path,
                     transform=transforms.Compose([    
                         transforms.RandomHorizontalFlip(),
                         transforms.ColorJitter(brightness=0.3),
                         transforms.RandomRotation(degrees=(-10, 10)),
                         transforms.ToTensor(),                  
                         transforms.Resize((448,448)),
                         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                  
                     ]))

# DataLoader
train_loader = DataLoader(train_dataset,
                        batch_size=batch_size,
                        shuffle=True)

In [6]:
# test 데이터셋 불러오기

test_path = '/aiffel/aiffel/final_project/dataset/fine_grained_data_tree/test'

# ImageFolder를 이용해 폴더명으로 class 지정
test_dataset = ImageFolder(root=test_path,
                     transform=transforms.Compose([
                         transforms.ToTensor(),                  
                         transforms.Resize((448,448)),
                         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                   
                     ]))

# DataLoader
test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=True)

In [7]:
# valid 데이터셋 불러오기

valid_path = '/aiffel/aiffel/final_project/dataset/fine_grained_data_tree/val'

# ImageFolder를 이용해 폴더명으로 class 지정
valid_dataset = ImageFolder(root=valid_path,
                     transform=transforms.Compose([       
                         transforms.ToTensor(),                  
                         transforms.Resize((448,448)),
                         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                  
                     ]))

# DataLoader
valid_loader = DataLoader(valid_dataset,
                        batch_size=batch_size,
                        shuffle=True)

In [11]:
# train_dataset

Dataset ImageFolder
    Number of datapoints: 9600
    Root location: /home/linkaiffel/dataset/fine_grained_tree/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ColorJitter(brightness=[0.7, 1.3], contrast=None, saturation=None, hue=None)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Resize(size=(448, 448), interpolation=bilinear, max_size=None, antialias=None)
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

### DFL에 적용할 모듈 import

In [19]:
model_vgg16 = dfl_model.DFL_VGG16(k = 10, nclass = 4)

In [20]:
model_vgg16

DFL_VGG16(
  (conv1_conv4): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d

In [21]:
model_vgg16.to(device)

DFL_VGG16(
  (conv1_conv4): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d

In [22]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model_vgg16.parameters(), lr=lr)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=20, T_mult=2, eta_min=1e-5)

In [23]:
train_config = {}
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch

train_config['Loss fn'] = str(criterion)[:-2]
train_config['Optimizer'] = str(optimizer).split()[0]
train_config['LR Scheduler'] = str(scheduler).split('.')[-1].split()[0]

### train, valid 함수 정의

In [24]:
# Train

def model_train(model, data_loader, criterion, optimzer, device):
    model.train() # 훈련 모드로 설정. training mode일 때 Gradient가 업데이트 됨
    losses, losses1, losses2, losses3, acc = 0 
    progress_bar = tqdm(data_loader)
    
    # mini-batch 학습 시작
    for img, label in progress_bar:
        img, label = img.to(device), label.to(device) # image, label 데이터를 device에 올림
        
        # 누적 Gradient 초기화
        optimizer.zero_grad()
        
        # foward propagation
        out1, out2, out3, _ = model(img)
        
        out = out1 + out2 + 0.1 * out3
        
        # 손실함수 계산
        loss1 = criterion(out1, label)
        loss2 = criterion(out2, label)
        loss3 = criterion(out3, label)
        
        loss = loss1 + loss2 + 0.1 * loss3
        
        # back propagation
        loss.backward()
        
        # Gradient 업데이트
        optimizer.step()
        
        # loss 업데이트
        losses1 += loss1.item() * img.size(0) / len(data_loader.dataset) # 배치 전체의 loss / dataset 전체 길이
        losses2 += loss2.item() * img.size(0) / len(data_loader.dataset)
        losses3 += loss3.item() * img.size(0) / len(data_loader.dataset)
        
        losses += loss.item() * img.size(0) / len(data_loader.dataset)
        
        # accuracy 업데이트
        pred = out.argmax(dim=1)
        acc += pred.eq(label).sum().item() # 정확히 맞춘 label의 개수
    print("train ===== " , acc, len(data_loader.dataset))   
    accuracy = acc / len(data_loader.dataset)
    
    return losses, losses1, losses2, losses3, accuracy

In [ ]:
# Validation

def model_evaluate(model, data_loader, criterion, device):
    model.eval()
    
    with torch.no_grad(): 
        
        val_loss, acc = 0
        examples = []

        for b_idx, (img, label) in tqdm(enumerate(data_loader), total=len(data_loader)):
            img, label = img.to(device), label.to(device)
            
            output1, output2, output3, _ = model(img)
            output = output1 + output2 + 0.1 * output3
            
            acc += torch.sum(output.argmax(dim=1).eq(label)).item()
            val_loss += criterion(output, label).item() * img.size(0)

            # wandb 에 이미지 시각화
            if b_idx < 4 : 
                imagenet_mean, imagenet_std = np.array([0.485,0.456,0.406]), np.array([0.229, 0.224, 0.225])
                for idx in range((img.shape[0])):
                    pixel = img[idx]
                    pixel = pixel.detach().cpu().numpy()
                    pixel = np.transpose(pixel, (1, 2, 0))
                    pixel = np.clip(255.0 * (pixel * imagenet_std + imagenet_mean), 0, 255)
                    pred = data_loader.dataset.classes[output.argmax(dim=1)[idx]]
                    target = data_loader.dataset.classes[label[idx]]
                    image = wandb.Image(pixel, caption=f'Pred : {pred}, Target : {target}')
                    examples.append(image)

            pred = output.argmax(dim=1)
            target_tensor = label
            pred_tensor = pred
            pred_tensor = torch.cat((pred_tensor, pred), dim=0)
            target_tensor = torch.cat((target_tensor, label), dim=0)

        f1 = metrics.F1Score(task="multiclass", num_classes=4).to(device)
        val_loss = val_loss / len(data_loader.dataset)
        accuracy = acc / len(data_loader.dataset)
        f1_score = f1(pred_tensor, target_tensor)
        print("valid ====== ",acc, len(data_loader.dataset))

    return val_loss, accuracy ,f1_score.cpu().item(), examples

In [27]:
wandb.init(project='[Classification]', name=f'{model_vgg16.__class__.__name__}', config=train_config, save_code=False)

min_loss = 1.0

for epoch in range(num_epoch):
    train_loss, train_loss1, train_loss2, train_loss3, train_acc = model_train(model_vgg16, train_loader, criterion, optimizer, device)
    val_loss, val_acc, f1_score, examples = model_evaluate(model_vgg16, valid_loader, criterion, device)
    
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(model_vgg16.state_dict(), f'./{model_vgg16.__class__.__name__}_Model.pth')
        
    print(f'epoch {epoch+1:02d}, train_loss: {train_loss:.5f}, train_accuracy: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}, f1_score: {f1_score:.5f}')

    wandb.log({'train_loss': train_loss, 'train_accuracy': train_acc, 'val_loss': val_loss, 'val_accuracy': val_acc,'f1_score': f1_score, 'examples': examples})
wandb.finish()    

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: seonwoooo. Use `wandb login --relogin` to force relogin


100%|██████████| 2400/2400 [14:01<00:00,  2.85it/s]


train =====  3832 9600


100%|██████████| 300/300 [00:51<00:00,  5.86it/s]


valid ======  563 1200
[INFO] val_loss has been improved from 1.20000 to 1.06663. Saving Model!
epoch 01, train_loss: 2.60268, train_accuracy: 0.39917, val_loss: 1.06663, val_accuracy: 0.46917


...

100%|██████████| 2400/2400 [14:07<00:00,  2.83it/s]


train =====  8291 9600


100%|██████████| 300/300 [00:52<00:00,  5.69it/s]

valid ======  899 1200
epoch 100, train_loss: 0.86529, train_accuracy: 0.86365, val_loss: 0.87337, val_accuracy: 0.74917


train_accuracy,▁▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
train_loss,█▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▁▃▂▄▄▄▅▅▅▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇█▇▇▇██▇███████
val_loss,█▇▅▅▅▄▄▄▄▃▃▃▄▃▄▃▄▃▄▂▄▂▄▂▁▃▁▃▄▂▃▃▅▃▄▄▆▅▄▄
train_accuracy,0.86365
train_loss,0.86529
val_accuracy,0.74917
val_loss,0.87337
